#### 一. 何为公平调度器
* Fair Scheduler是一种随着时间增长, app之间平均获取资源的调度方法
* 默认情况下, 只基于memory左公平资源分配, 可通过配置支持多资源的公平调度(memory和cpu), 此时采用[DRF主资源公平调度](https://cs.stanford.edu/~matei/papers/2011/nsdi_drf.pdf)(Dorminant Resources Fairness)
* 公平调度, 并非所有app平均共享资源, 而是通过把app提交到各个队列, 对每个队列指定分3个主要属性: queue分配资源的权重, queue分配的最小资源, queue分配的最大资源. 每个queue内的app平均的分配资源, 但每个queue占用的资源不一样.
* 初始情况下,没有app的queue不会占用资源, 有app的queue会占用集群剩余资源; 当1个app提交到queue时,该app会等待其他queue的app释放资源, 然后逐步分配到该queue的最小资源配额, 当然如果该app用不到最小资源配额, 也是可以的
* 如果queue的Used Resources小于Instantaneous Fair Share, 则该queue的资源不能被其他queue抢占(preemption), 因为该queue占用的资源还未到其权重限制后的可占资源. 其次, 队列还可以配置可同时运行的app数量, 用户使用权限等属性

<img src='img/fair.png'>

#### 二. Steady share计算方法
* Steady share, 是根据权重,最大资源,最小资源的配置, 计算出的期望这个队列能从集群中获得多少资源
* 首先来定义一个概念: 加权公平性 `(weighted fair sharing)`在`max share`和`min share`存在的情况下.   
如果没有`max share`和`min share`, 加权平均的意思为:对于所有queue, 其单位资源是相等的, 即$$单位资源=\frac{queue分到的资源  }{queue配置的权重}$$但是, 当有了`max share`和`min share`后, 每个queue分到的资源可能会比`max share`还大或者比`min share`还小, 这个问题需要被解决  
[To understand what this method does, we must first define what weighted fair sharing means in the presence of min and max shares. If there were no minimum or maximum shares, then weighted fair sharing would be achieved if the ratio of slotsAssigned / weight was equal for each Schedulable and all slots were assigned. Minimum and maximum shares add a further twist - Some Schedulables may have a min share higher than their assigned share or a max share lower than their assigned share.]()

* 为了解决这个问题, 定义一比值$R$(权重=1时的资源) , 对于每个可调度的队列$S$   
    * 当`R*S.weight>S.maxShare`时, 队列S被分到的资源为`S.maxShare`; 
    * 当`R*S.weight<S.minShare`时, 队列S被分到的资源为`S.minShare`;
    * 其他情况被分到的资源为`R*S.weight`  
[To deal with these possibilities, we define an assignment of slots as being fair if there exists a ratio R such that:   
Schedulables S where S.minShare {@literal >} R * S.weight are given share S.minShare   
Schedulables S where S.maxShare {@literal <} R * S.weight are given S.maxShare   
All other Schedulables S are assigned share R * S.weight - The sum of all the shares is totalSlots.]()
    * 把比值R称为`weight-to-slots`比率, 因为比值R把queue的权重转换为其实际被分配到的资源(slot)

* 如何找到比值R
    * 为了找到比值R, 采用二分查找法. 假设给定一个R, 计算所有队列可分配到的资源. 使用第二点的条件, 计算和$$\sum{(R*S.weight) or (minShare) or (maxShare)}$$如果这个和比总资源小, 则R取值偏小; 如果这个和比总资源大, 则R取值偏大;
    * 二分法的起始边界为:
        * 0作为R取值的下界, 这意味着所有queue只能被分到minShare, 因为0\*weight=0, 一定小于minShare
        * R的上界通过迭代计算求得, 每次对R翻倍, 直到计算所得的所有queue分得的资源数量, 比总资源数量要大为止        
        
    [We compute a fair allocation by finding a suitable weight-to-slot ratio R. To do this, we use binary search. Given a ratio R, we compute the number of slots that would be used in total with this ratio (the sum of the shares computed using the conditions above). If this number of slots is less than totalSlots, then R is too small and more slots could be assigned. If the number of slots is more than totalSlots, then R is too large.  
We begin the binary search with a lower bound on R of 0 (which means that all Schedulables are only given their minShare) and an upper bound computed to be large enough that too many slots are given (by doubling R until we use more than totalResources resources). The helper method resourceUsedWithWeightToResourceRatio computes the total resources used with a given value of R.]()  
    * 该算法最多迭代25次, 来找到合适的比值R

#### 三. Instantaneous share计算方法
* Instantaneous share计算方法和Steady share计算方法一致, 调用的一个方法.
* 只是Steady share只在启动时计算一次, 或在addNode时计算; 而Instantaneous share 是有一个线程在后台定时执行, 且只计算存在app的queue, 不存在app的queue不用与计算$\sum{(R*S.weight) or (minShare) or (maxShare)}$